# Migration script

Migrate the input databases from the 2.29 format to the 2.31.1 format.

This notebook assumes `SCENARIO_TO_MIGRATE` points to a scenario of Kleinalbis (which buildings?) in v2.29.0 format and `EXAMPLE_SCENARIO` to a scenario in v2.31.1 format. 

These are the v2.29.0 format files to migrate:

- shapefiles 
  - `inputs/building-geometry/zone.shp`... checked. no changes!
  - `inputs/building-geometry/site.shp`... checked. no changes!
  - `inputs/building-geometry/surroundings.shp`... checked. no changes!
- inputs to archetypes-mapper
  - `inputs/building-properties/age.dbf`... checked: merge to `typology.dbf`
  - `inputs/building-properties/occupancy.dbf`... checked: merge to `typology.dbf`
- outputs of archetypes-mapper
  - `inputs/building-properties/air_conditioning_systems.dbf`
  - `inputs/building-properties/architecture.dbf`
  - `inputs/building-properties/indoor_comfort.dbf`
  - `inputs/building-properties/internal_loads.dbf`
  - `inputs/building-properties/supply_systems.dbf`
  - `inputs/building-properties/schedules/B*.csv`... checked: add ELECTROMOBILITY column?

Also, remember, the inputs/technology folder also changed and the v2.31.1 format references these possibly differently.

### Assumptions

To make this easier, here are some assumptions this script makes:

- use types in `occupancy.dbf` correspond to use types in `USE_TYPE_PROPERTIES.XSLX` (and corresponding `*.csv` schedules)
- user edits the schedules in the database selected, unless user has defined their own schedules with the `schedule-maker` tool - in that case, don't create schedules with the `archetypes-mapper`!
- no more than 3 uses per row in `occupancy.dbf`

In [95]:
import os
import cea
import geopandas
import pandas as pd
from packaging import version

import cea.inputlocator
from cea.utilities.dbf import dbf_to_dataframe, dataframe_to_dbf

In [96]:
# Constants
SCENARIO_TO_MIGRATE = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\2.29.0\kleinalbis"
EXAMPLE_SCENARIO = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\working-with-databases\kleinalbis"

# the (new) database to use for migration - user should prepare this in advance...
DATABASE = r"c:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\databases\CH"
CONSTRUCTION_STANDARD_XLSX = os.path.join(DATABASE, "archetypes", "CONSTRUCTION_STANDARD.xlsx")

In [97]:
# we can't use the input locator for the v2.29.0 scenario, but we _can_ (and should) use it for v2.31.1.
locator = cea.inputlocator.InputLocator(scenario=EXAMPLE_SCENARIO)

## Migrate building-geometry files

In [98]:
# let's first compare the two files (and practice reading/writing shape files)
old_zone_shp = geopandas.GeoDataFrame.from_file(
    os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-geometry", "zone.shp"))
old_zone_shp

,Name,height_ag,floors_ag,height_bg,floors_bg,descriptio,category,REFERENCE,geometry
0,B1000,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."
1,B1001,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462736.5184668251 15245045.1003448, ..."
2,B1002,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462730.7804993654 15245054.87234257,..."
3,B1003,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462743.7118557409 15245049.30022802,..."
4,B1004,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462737.9663290229 15245059.07226665,..."
5,B1005,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."


In [99]:
new_zone_shp = geopandas.GeoDataFrame.from_file(locator.get_zone_geometry())
new_zone_shp

,Name,height_ag,floors_ag,height_bg,floors_bg,descriptio,category,REFERENCE,geometry
0,B1000,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."
1,B1001,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462736.5184668251 15245045.1003448, ..."
2,B1002,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462730.7804993654 15245054.87234257,..."
3,B1003,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462743.7118557409 15245049.30022802,..."
4,B1004,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462737.9663290229 15245059.07226665,..."
5,B1005,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."


The zone.shp file is the the same! (repeated this procedure with site.shp and surroundings.shp)

## Migrate age.dbf file and occupancy.dbf file

In [100]:
old_age_dbf = dbf_to_dataframe(os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-properties", "age.dbf"))
old_age_dbf

,HVAC,Name,REFERENCE,basement,built,envelope,partitions,roof,windows
0,0,B1000,OSM - as it is,0,1943,0,0,0,0
1,0,B1001,OSM - as it is,0,1943,0,0,0,0
2,0,B1002,OSM - as it is,0,1943,0,0,0,0
3,0,B1003,OSM - as it is,0,1943,0,0,0,0
4,0,B1004,OSM - as it is,0,1943,0,0,0,0
5,0,B1005,OSM - as it is,0,1943,0,0,0,0


In [101]:
# i think that should be migrated to typology.dbf
new_typology_dbf = dbf_to_dataframe(locator.get_building_typology())
new_typology_dbf

,1ST_USE,1ST_USE_R,2ND_USE,2ND_USE_R,3RD_USE,3RD_USE_R,Name,REFERENCE,STANDARD,YEAR
0,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1000,OSM - as it is,STANDARD2,1943
1,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1001,OSM - as it is,STANDARD2,1943
2,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1002,OSM - as it is,STANDARD2,1943
3,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1003,OSM - as it is,STANDARD2,1943
4,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1004,OSM - as it is,STANDARD2,1943
5,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1005,OSM - as it is,STANDARD2,1943


OK. So. This is going to be interesting.

- old_age_dbf.Name --> new_typology_dbf.Name
- old_age_dbf.built --> new_typology_dbf.YEAR

(I'm assuming we just forget the other stuff for now - maybe when we look into the old technologies folder, we'll figure out a way to translate the HVAC, basement, envelope, partitions, roof and windows fields. Maybe.)

In [102]:
old_occupancy_dbf = dbf_to_dataframe(
    os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-properties", "occupancy.dbf"))
# old_occupancy_dbf

Clearly, this information goes into the `typology.dbf` file - we just need to figure out how to map the uses.

In [117]:
# NOTE: this stuff is good for the migration script


def lookup_standard(year, standards_df):
    # find first standard that is similar to the year
    standard = standards_df[(standards_df.YEAR_START < year) & (year < standards_df.YEAR_END)].iloc[0]
    return standard.STANDARD

def convert_occupancy(name, occupancy_dbf):
    # FIXME: do this smorter
    row = occupancy_dbf[occupancy_dbf.Name == name].iloc[0]
    uses = set(row.to_dict().keys()) - set(["Name", "REFERENCE"])
    uses = sorted(uses, cmp=lambda a, b: cmp(float(row[a]), float(row[b])), reverse=True)
    result = {
        "use_type1": uses[0], 
        "use_type1r": float(row[uses[0]]), 
        "2ND_USE": uses[1], 
        "2ND_USE_R": float(row[uses[1]]), 
        "3RD_USE": uses[2], 
        "3RD_USE_R": float(row[uses[2]])}
    if pd.np.isclose(result["2ND_USE_R"], 0.0):
        result["1ST_USE_R"] = 1.0
        result["2ND_USE_R"] = 0.0
        result["3RD_USE_R"] = 0.0
        result["2ND_USE"] = "NONE"
        result["3RD_USE"] = "NONE"
    elif pd.np.isclose(result["3RD_USE_R"], 0.0):
        result["1ST_USE_R"] = 1.0 - result["2ND_USE_R"]
        result["3RD_USE_R"] = 0.0
        result["3RD_USE"] = "NONE"
    
    result["1ST_USE_R"] = 1.0 - result["2ND_USE_R"] - result["3RD_USE_R"]
    return result

def merge_age_and_occupancy_to_typology(age_dbf, occupancy_dbf, standards_df):
    # merge age.dbf and occupancy.dbf to typology.dbf
    typology_dbf_columns = ["Name", "YEAR", "STANDARD", "1ST_USE", "1ST_USE_R", "2ND_USE", "2ND_USE_R", "3RD_USE", "3RD_USE_R"]
    typology_dbf = pd.DataFrame(columns=typology_dbf_columns)
    
    for rindex, row in age_dbf.iterrows():
        typology_row = {
            "Name": row.Name, 
            "YEAR": row.built,
            "STANDARD": lookup_standard(row.built, standards_df)}
        typology_row.update(convert_occupancy(row.Name, occupancy_dbf))
        
        typology_dbf = typology_dbf.append(typology_row, ignore_index=True)
    
    return typology_dbf


standards_df = pd.read_excel(CONSTRUCTION_STANDARD_XLSX, "STANDARD_DEFINITION")
typology_dbf = merge_age_and_occupancy_to_typology(old_age_dbf, old_occupancy_dbf, standards_df)
typology_dbf

,Name,YEAR,STANDARD,1ST_USE,1ST_USE_R,2ND_USE,2ND_USE_R,3RD_USE,3RD_USE_R
0,B1000,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0
1,B1001,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0
2,B1002,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0
3,B1003,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0
4,B1004,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0
5,B1005,1943,STANDARD2,SINGLE_RES,1.0,NONE,0.0,NONE,0.0


## Migrate inputs/building-properties/air_conditioning_systems.dbf

- copy Name, cool_ends, cool_starts, heat_ends, heat_starts
- look up type_cs, type_ctrl, type_dhw, type_hs, type_vent (this is cool - maybe only the names changed?)
- type_dhw: looks up (Tsww0_C, Qwwmax_Wm2) from database/assemblies/HVAC.xls
- type_hs: trickier... i don't know if this can be done automatically?

My thinking right now is to use the `archetypes-mapper` tool to do the rest of the migration...

In [119]:
air_conditioning_systems_dbf = dbf_to_dataframe(
    os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-properties", "air_conditioning_systems.dbf"))
air_conditioning_systems_dbf

,Name,cool_ends,cool_starts,heat_ends,heat_starts,type_cs,type_ctrl,type_dhw,type_hs,type_vent
0,B1000,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0
1,B1001,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0
2,B1002,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0
3,B1003,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0
4,B1004,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0
5,B1005,15|09,15|05,14|05,16|09,T0,T1,T1,T1,T0


In [120]:
dbf_to_dataframe(locator.get_building_air_conditioning())

,Name,cool_ends,cool_starts,heat_ends,heat_starts,type_cs,type_ctrl,type_dhw,type_hs,type_vent
0,B1000,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0
1,B1001,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0
2,B1002,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0
3,B1003,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0
4,B1004,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0
5,B1005,15|09,15|05,14|05,16|09,HVAC_COOLING_AS0,HVAC_CONTROLLER_AS1,HVAC_HOTWATER_AS1,HVAC_HEATING_AS1,HVAC_VENTILATION_AS0


In [121]:
locator.get_database_construction_standards()

'c:\\Users\\darthoma\\Documents\\CityEnergyAnalyst\\projects\\working-with-databases\\kleinalbis\\inputs\\technology\\archetypes\\CONSTRUCTION_STANDARD.xlsx'